In [1]:
import pandas as pd

In [2]:
txt = open("Marco.rtf",newline = '\r').read()

In [3]:
#Formatando as linhas
txt = (txt.split(sep = '\n')) 

In [4]:
df = pd.DataFrame()
df['col'] = txt

In [5]:
#Separando creditos e debitos
credits, debits, credits_desc, debits_desc = [],[],[],[]

for i in range(len(df)-1):
    
    if df.col[i][-1] == 'C' and 'SALDO DO DIA' not in df.col[i]:
        credits.append(df.col[i])
        credits_desc.append(df.col[i+1])
    elif df.col[i][-1] == 'D':
        debits.append(df.col[i])
        debits_desc.append(df.col[i+1])

### Organizando créditos

In [6]:
def separa_valor(df,df_col,colname):
    '''Uma funçao que separa os valores contidos em uma unica coluna'''
    
    if colname == 'cred':
        val = 2
    elif colname == 'debt':
        val = 3
    
    #Cria novas colunas para a data e valor de cada transação
    data,valor = [],[]
    for i in range(len(df)):
        data.append(df_col[i].rsplit()[0])
        valor.append(df_col[i].rsplit()[val])
        
    df['data'] = data
    df['valor'] = valor
    df.drop(columns = colname, axis =1, inplace = True)
    
    return df

In [7]:
def formata_df(df,strip):
    df['valor'] = df['valor'].str.replace(strip,'')
   # df['valor'] = df['valor'].str.replace(',','.')
    if strip == 'C':
        df = df[['data','cred_desc','valor']]
    elif strip == 'D':
        df = df[['data','debt_desc','valor']]
        
    for i in range(len(df)):
        df.iloc[i][0] = df.iloc[i][0][2:] + '/' + df.iloc[i][0][:2]
    
    return df

In [8]:
#Criando dataframe para armazenar creditos e suas descrições
df_cred = pd.DataFrame()
df_cred['cred'] = credits
df_cred['cred_desc'] =credits_desc

In [9]:
df_cred = separa_valor(df_cred,df_cred.cred,'cred')

In [10]:
#Limpando rows indesejadas
df_cred = df_cred[df_cred.data != 'SALDO']
df_cred = df_cred[df_cred.data != 'LIMITE']
df_cred = df_cred[df_cred.data != 'RDC']
df_cred = df_cred[df_cred.valor != 'ANTERIOR']
df_cred = df_cred[df_cred.valor != 'CRÉD.DEVOLUÇÃO']

In [11]:
df_cred = formata_df(df_cred,'C')

In [12]:
df_cred.to_excel('creditos_new.xlsx', index=False)

### Organizando débitos 

In [13]:
#Criando dataframe para armazenar creditos e suas descrições
df_debt = pd.DataFrame()
df_debt['debt'] = debits
df_debt['debt_desc'] =debits_desc

In [14]:
#A função separa_valor só funciona para quando a row tem 4 palavras, essa célula garante que a row tenha 4 palavras
for i in range(len(df_debt.debt)):
    
    if len(df_debt.debt[i].rsplit()) >4:
        delet = len(df_debt.debt[i].rsplit()) -4
        for j in range(delet):
            df_debt.debt[i] = df_debt.debt[i].strip(str(df_debt.debt[i].split(' ', 1)[j]))
            
    if len(df_debt.debt[i].rsplit()) <4:
        add = 4 - len(df_debt.debt[i].rsplit()) 
        df_debt.debt[i] = ('0 '*add)+df_debt.debt[i]

In [15]:
#Limpando rows indesejadas
lista = ["PREVISÃO CPMF","PREVISÃO IOF","PREVISÃO ENCARGOS","PREVISÃO TARIFAS"]
for i in lista:
    df_debt = df_debt[~df_debt.debt.str.contains(i)]

In [16]:
df_debt = separa_valor(df_debt,df_debt.debt,'debt')

In [17]:
df_debt = formata_df(df_debt,'D')

In [18]:
df_debt.to_excel('debitos_new.xlsx', index=False)